# Linear Transformations
---
**Name:** Jonh Alexis Buot <br>
**Date:** December 2023 <br>
**Course:** CS3101N <br>
**Task:** Assignment - Linear Transformations

---

In [1]:
import random
import numpy as np

# Code Challenges

**1. Develop a python function from scratch that will find the determinants of any $n$ x $n$ matrix.**

In [2]:
def mat_det(A):
# =============================================================================
#   Calculate the determinant of a square matrix A.
# =============================================================================
    assert len(A) == 0 or all(type(row) == list for row in A) and len(A) == len(A[0]), "A should be a square matrix."
    
    n = len(A)
    if   n == 0: return 1.
    elif n == 1: return A[0][0]
    elif n == 2: return A[0][0] * A[1][1] - A[0][1] * A[1][0] 

    det = 0.
    for col, cofactor in enumerate(A[0]):
        minor = [[A[i][j] for j in range(n) if j != col] for i in range(1, n)]
        det += (-1) ** col * cofactor * mat_det(minor)

    return det

**2. Develop a python function from scratch that will find both the eigenvectors and eigenvalues of any $n$ x $n$ matrix.**

In [387]:
def dot(vec_a, vec_b):
    return sum(a * b for a, b in zip(vec_a, vec_b))

def normalize(vector):
    vec_norm = sum(a ** 2 for a in vector) ** 0.5
    return [entry / vec_norm for entry in vector]

def transform(matrix, vector):
    return [sum(row_entry * vector_entry for row_entry, vector_entry in zip(row, vector)) for row in matrix]

def mat_eig(A, max_iterations = 50000):
# =============================================================================
#   Calculate the eigenvectors and eigenvalues of a square matrix A.
#   
#   Note: This uses power method with Weilandt deflation.
#   https://services.math.duke.edu/~jtwong/math361-2019/lectures/Lec10eigenvalues.pdf
#   https://www.math.umd.edu/~rvbalan/TEACHING/AMSC663Fall2015/PROJECTS/P2/AMSC663664%20Midyear%20Report_DanielleMiddlebrooks.pdf
# =============================================================================
    assert len(A) == 0 or all(type(row) == list for row in A) and len(A) == len(A[0]), "A should be a square matrix."

    n = len(A) 
    if n == 0: return [], []
    if n == 1: return [A[0][0]], [[1.]]

    eigenvectors = []
    eigenvalues = []

    eigvec = normalize([random.random() for _ in range(n)])
    
    for _ in range(n):
        # Power Method
        for _ in range(max_iterations):
            eigvec = transform(A, eigvec)
            eigvec = normalize(eigvec) 

            tf_eigvec = transform(A, eigvec) 
            eigval = dot(eigvec, tf_eigvec)
            
        eigenvalues.append(eigval)
        eigenvectors.append(eigvec)

        # Deflate Matrix with Weilandt deflation
        outer_product = [[eigval * a * b for b in eigvec] for a in eigvec]
        A = [[A[i][j] - outer_product[i][j] for j in range(n)] for i in range(n)]

        guess_eigvec = normalize([random.random() for _ in range(n)])
        proj_factor = dot(eigvec, guess_eigvec) / dot(eigvec, eigvec)

        eigvec = [entry - proj_factor * entry for entry in guess_eigvec]

    return eigenvalues, [list(row) for row in zip(*eigenvectors)]

**3. Test your functions from a randomly generated $n$ x $n$ matrix.**

- Testing and comparing my determinant implementation to the `numpy` implementation for an $n$ x $n$ matrix.

In [397]:
A = np.random.random((8, 8)).astype("float64") * 5

my_result = mat_det(A.tolist())
np_result = np.linalg.det(A)

print("-- A --", A, sep="\n")
print("\n-- My det(A) --", my_result, sep="\n")
print("\n-- Numpy det(A) --", np_result, sep="\n")
print("\nAre my results equal to numpy?", "Yes." if np.isclose(my_result, np_result) else "No.")

-- A --
[[4.98390461 1.63467746 3.44200947 0.86074013 2.55808818 3.94359729
  4.4015802  1.76228538]
 [3.746334   4.77054613 4.90186545 3.18545827 0.89319034 1.67814781
  1.26443645 0.94101771]
 [2.71471303 0.07278666 2.06979264 3.0263978  4.68253045 0.15064801
  2.31888178 3.89684038]
 [4.35286512 4.76923123 0.28189721 3.80366929 3.83473056 0.50710951
  2.8496769  1.44115033]
 [2.78025046 0.01633914 0.85254477 0.49796835 3.09508406 1.03865323
  0.49482846 1.20550739]
 [3.31143881 4.03251604 0.06407929 0.52017303 1.59731948 3.31809417
  0.72305312 0.54768307]
 [4.10947835 1.30649059 0.11370461 3.1921519  2.95633039 4.40659993
  4.3994732  2.15106668]
 [0.72316552 2.70979877 4.59044597 3.29480089 3.53860311 0.15506324
  3.32601769 0.07167396]]

-- My det(A) --
42151.95980819416

-- Numpy det(A) --
42151.95980819413

Are my results equal to numpy? Yes.


- Testing and comparing my eigenvalue and eigenvector algorithm implementation to the `numpy` implementation for an $n$ x $n$ matrix.

In [386]:
A = np.random.random((3, 3)).astype("float64") * 5

np_ev, np_evl = np.linalg.eig(A)
m_ev, m_evl = mat_eig(A.tolist())
m_ev, m_evl = np.array(m_ev), np.array(m_evl)

print("-- A --", A, sep="\n")
print("\n-- My Eigenvalues and Eigenvectors --", m_ev, m_evl, sep="\n")
print("\n-- Numpy Eigenvalues and Eigenvectors --", np_ev, np_evl, sep="\n")

print("\n\n", "Are the differences of eigenvalues close?", "Yes." if np.allclose(m_ev, np_ev) else "No.")

cosine_similarity = np.array([npe.dot(me) / (np.linalg.norm(me) * np.linalg.norm(npe)) for npe, me in zip(np_evl.T, m_evl.T)]).T
print("\n-- Cosine Similarity of Eigenvectors --", cosine_similarity, sep="\n")

-- A --
[[3.28600296 1.64424693 1.17217275]
 [0.74998807 0.63806035 0.40745403]
 [0.62328301 2.49202929 1.36374031]]

-- My Eigenvalues and Eigenvectors --
[ 4.23051027e+00  1.06103232e+00 -3.73897656e-03]
[[ 0.88985175 -0.21680576  0.20128907]
 [ 0.23043492  0.07476357 -0.07826623]
 [ 0.39378118  0.97334766 -0.97640008]]

-- Numpy Eigenvalues and Eigenvectors --
[ 4.23051027e+00  1.06103232e+00 -3.73897656e-03]
[[-0.88985175 -0.47249637 -0.08272621]
 [-0.23043492  0.011129   -0.46344065]
 [-0.39378118  0.88126235  0.88225798]]


 Are the differences of eigenvalues close? Yes.

-- Cosine Similarity of Eigenvectors --
[-1.          0.96104662 -0.84181689]
